<a href="https://colab.research.google.com/github/ovr4/NIST/blob/main/HTML_Table_Extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [313]:
#pip install bleach

In [314]:
#pip install beautifulsoup4

In [315]:
pip install pint

In [316]:
pip install units

This program extracts tabular data from concrete research articles

In [317]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import bleach
import re
import numpy as np

In [318]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The class HTMLTableParser extracts the contents of tables and returns a pandas dataframe of the table, the headers and two layers of sub headers from the table to guide aggregation in a later step

In [319]:
'''VARIABLES SPECIFIED BY USERS'''


##Materials to determine compositions of 
prop_dict = {'Cement': ['Cement', 'OPC', 'ASTM', 'P.O', 'P·O', 'cement'], 'Limestone': ['Limestone', 'Limestone filler', 'LF'], 
                'Fly ash': ['FA', 'FAM', 'Fly Ash', 'Fly ash'], 'Slag': ['Slag', 'SL'],
                'Water': ['Water', 'H2O'], 'Quartz powder': ['Quartz powder'],
                'Sand': ['Quartz sand', 'Sand', 'Fine aggregates', 'sand'],
                'Coarse': ['Coarse aggregates', 'Coarse '], 'Metakaolin': ['MK'],
                'Silica': ['Silica', 'Silica fume']}
prop_dict = {'Cement': ['Cement', 'OPC', 'ASTM', 'P.O', 'P·O', 'cement'], 'Limestone': ['Limestone', 'Limestone filler', 'LF'], 
            'Fly ash': ['FA', 'FAM', 'Fly Ash', 'Fly ash'], 'Slag': ['Slag', 'SL'],
            'Water': ['Water', 'H2O'], 'Quartz powder': ['Quartz powder'],
            'Sand': ['Quartz sand', 'Sand', 'Fine aggregates', 'sand', 'aggregate'],
            'Coarse': ['Coarse aggregates', 'Coarse ', 'Aggregate'], 'Metakaolin': ['MK'],
            'Silica': ['Silica', 'Silica fume'], 'Other': ['Paste']}

prop_dict = {'Cement': ['Cement', 'OPC', 'ASTM', 'P.O', 'P·O', 'cement'], 'Limestone': ['Limestone', 'Limestone filler', 'LF'], 
                  'Fly ash': ['FA', 'FAM', 'Fly Ash', 'Fly ash'], 'Slag': ['Slag', 'SL'],
                  'Water': ['Water', 'H2O'], 'Quartz powder': ['Quartz powder'],
                  'Sand': ['Quartz sand', 'Sand', 'Fine aggregates', 'sand'],
                  'Coarse': ['Coarse aggregates', 'Coarse '], 'Metakaolin': ['MK'],
                  'Silica': ['Silica', 'Silica fume']}

### Final products to extract data from  
material = {'CONCRETE': ['SCC', 'Concrete', 'concrete', 'SCLC', 'HPC', 'UHPC'], 'PASTE': ['paste', 'Paste'], 'MORTAR': ['Mortar', 'mortar', 'grout', 'Grout']}

fin_mat_col = '/'.join(material.keys())
#print(fin_mat_col)
###Compositions of materials 
compounds_to_look_for = ['CaO', 'SiO2', 'Al2O3', 'Fe2O3', 
                            'MgO', 'K2O', 'Na2O', 'SO3', 'SrO', 'P2O5', 
                            'TiO2', 'MoO3',  'BaO', 'Cl', 'MnO', 'C3S', 
                            'C2S', 'C3A', 'C4AF', 'ZrO2', 'Cr2O3', 'CuO', 
                            'ZnO', 'Mn2O3', 'LOI', 'Loss on ignition']

compounds_to_look_for = ['CaO', 'SiO2', 'Al2O3', 'Fe2O3', 
                            'MgO', 'K2O', 'Na2O', 'SO3', 'SrO', 'P2O5', 
                            'TiO2', 'MoO3',  'BaO', 'Cl', 'MnO', 'C3S', 
                            'C2S', 'C3A', 'C4AF', 'ZrO2', 'Cr2O3', 'CuO', 
                            'ZnO', 'Mn2O3', 'LOI', 'Loss on ignition']
                            #, 'Specific gravity'
                            #'Specific surface', 'Setting time', 'Compressive '
                            #'Flexural', 'Flexural ', 'Density', 'surface area', 'SSD', 'OD'
                            #'Specific', 'Bulk density', 'Blaine', 'Specific gravity', 'sieve']

###List of all probable units for this subfield
from pint import UnitRegistry
ureg = UnitRegistry()

list_of_all_units = ['°C','kg', '%', 'ratio', 'kg/m2', 'kg/m3', 'mm', 'Pa', 'Pa.s', 'g', 'MPa', 'wt. %', 'h−1', 'k', 'min', 'hours', 'time', 'set time', '°C']
list_of_all_units = ['%', 'ratio', 'kg/m2', 'kg/m3', 'by cement weight', 'weight',
                          'mm', 'Pa', 'Pa.s', 'g', 'MPa', 'N', 'N.mm' 
                          'wt. %', 'h−1', 'kg/m', 'kg', 
                          'k', 'min', 'hours', 'time', 'set time', '°C', 'ηθ', 'sec', 'N.mm', 'm2', 'η', 'nm', 
                          'kN', 'm2/kg', 'μm', 'ml/s']

                               
add_ons = ['%', 'ratio', 'weight']

def cubic_square_converter(unit):
  for i, j in zip(range(2,4), ['square', 'cubic']):
    if str(i) in unit:
      og_unit = unit.split(str(i))[0]
      try:
        ureg[og_unit]
        new_unit = j+' '+og_unit
        unit = unit.replace(og_unit+str(i), new_unit)
        return unit 
      except:
        pass
    else:
      pass
  return unit


new_column = []
##print(unit, df_columns)
for i in list_of_all_units:
    new_column.append('('+cubic_square_converter(i)+')')
t = 1.0*ureg['kg/cubic m']
##print(cubic_square_converter('m2'))
##print(new_column)
for i in new_column:
  #unit = i.split('(')[-1].split(')')[0]
  #value = 1.0*ureg[unit]  
  try:
    unit = i.split('(')[-1].split(')')[0]
    value = 1.0*ureg[unit]
    ##print(value, unit)
  except:
    pass

keep = ['kg', '%', 'ratio', 'kg/m2', 'kg/m3', 'Pa', 'Pa.s', 'MPa', 'wt. %', 'h−1', 'min', '(min)', 'time', '°C', 'gravity']
keep = ['kg', '%', 'ratio', 'kg/m2', 'kg/m3', 'Pa', 'Pa.s', 'MPa', 'wt. %', 'h−1', 
            'min', '(min)', 'time', '°C', 'gravity']

l_units = []
for unit in list_of_all_units:
  l_units.append('('+unit+')')
  l_units.append('('+unit)
  if unit in keep:
    l_units.append(' '+unit)

##Header ID to look for i.e. the likely header name of unique mixtures in article
headers_to_look_for = ['code', 'ID', 'No.', 'Mix', 'Mixtures', 'Code', 'name', 'mix', 'Mixture']
headers_to_look_for = ['ID', 'No.', 'Mix', 'Mixtures', 'Code', 'name', 'mix', 'Mixture']
headers_to_look_for = ['ID', 'No.', 'Mix', 'Mixtures', 'Code', 'name', 'mix', 'Mixture', 'Material']

headers_to_look_for = ['Authors', 'system', 'code', 'ID', 'No.', 'Mix', 'Material', 'Mixtures', 'Code', 'name', 'mix', 'Mixture', 'Notation' , 'N ', 'batch', 'Label', 'label']
control_ids = ['Control', 'CO']

global headers_to_look_for, control_ids
global compounds_to_look_for
global prop_dict, material
global list_of_all_units, keep
###The properties to extract and their synonyms
##loaded from excel file


#BUILT IN

##Used to identify if data is a header or entry
breakers = [' ', '-', '%', '&', '–', '≥', '<', '±']



In [320]:
def convert_ids_to_string(df):
  for index, i in enumerate(df.columns):
    new_col  = []
    if any(header in i for header in headers_to_look_for):
      for j in df.iloc[:, index]:
        new_col.append(str(j))
      df.iloc[:, index] = new_col
  return df

def clean_table_headers(columns):
    ##clean column units nicely
    new_head = []
    possible_unit_format = [', ', '-']
    for i in columns:
      got_it_a, got_it_b = False, False     
      i = i.replace('BLANK HEADER/', '')
      try:
        unit = i.split(')')[0].split('(')[-1]
        for format in possible_unit_format:
          if format+unit in i and '('+unit+')' in i:# and not got_it_a:
            new_name = i.replace(format+unit, '')
            got_it_a = True
            break
          elif int(common('('+unit+')', i, get_length=True)) > 1:# and not got_it_b:
            if '('+unit+')/' in i:
              new_name = i.replace('('+unit+')/', '/')
              got_it_b = True
              break
        if got_it_b or got_it_a:      
          new_head.append(new_name)
        else:
          new_head.append(i)
      except:        
          new_head.append(i)
    new_head = remove_math_symbols(new_head)
    final_header = []
    for i in new_head:
      if '()' in i:
        i = i.replace('()', '')
      final_header.append(i)
    ##print(final_header)
    return final_header

def remove_math_symbols(columns):
  new_columns = []
  for col in columns:
    o_col = col
    col = str(col)
    if '<math>' in col:
      new_col = col.split('<math>')[0]
      unit = col.split('</math>')[-1]
      new_columns.append(new_col+' '+unit)
    else:
      new_columns.append(o_col)
  return new_columns

def concrete_mortar_paste(title): 
  
  ''' this function determines whether a table is reporting the concrete, mortar or paste properties'''

  '''    **Parameters**

        title: *str*
            The title of the table         
      
      **Returns**
      Label identifying the material being reported in the table: Concrete, mortar, or paste
  '''


  
  material_table_list = []
  
  if type(title) == list:
    for i in title:
      sum_key = ''
      got_it = False
      for keys in material.keys():
        if any(mat in i for mat in material[keys]):
          if keys not in sum_key:
            sum_key += keys+'/'
      
      if sum_key == '':
        material_table_list.append(None)
      else:
        if sum_key[-1] == '/':
          sum_key = sum_key[:-1]
        material_table_list.append(sum_key)
    return material_table_list
  
  elif type(title) == str:
    sum_key=''
    for keys in material.keys():
      if any(mat in title for mat in material[keys]):
        if keys not in sum_key:
          sum_key += keys+'/'
    if sum_key =='':  
      return None
    else:
      if sum_key[-1] == '/':
        sum_key = sum_key[:-1]
      return sum_key
  

def check_units(object_to_check, get_all_units_list=False):
  '''this function checks if there is unit in a column entry or title and returns the unit'''
  
  if get_all_units_list:
    if type(object_to_check) == str:

      for unit in list_of_all_units:
        if '('+unit+')' in object_to_check or ' '+unit+' ' in object_to_check or '/'+unit in object_to_check or ', '+unit in object_to_check:
          return True ## meaning that the column already has a unit

  else:
    if type(object_to_check) == list:
      unit_list = []
      
      for col in object_to_check:
        current_len_unit_list = len(unit_list)
        

        for unit in list_of_all_units:
          if '('+unit+')' in col or ' '+unit+' ' in col or '/'+unit in col or ', '+unit in col:
            unit_list.append(unit) 

        if current_len_unit_list == len(unit_list):
          unit_list.append('No unit')
      return unit_list
    
    elif type(object_to_check) == str:                  
        for unit in list_of_all_units:
          if '('+unit in object_to_check or '('+unit in object_to_check or '('+unit+')' in object_to_check or ' '+unit+' ' in object_to_check or '/'+unit in object_to_check or ', '+unit in object_to_check:
            return unit
        if '(' in object_to_check and ')' in object_to_check:
          unit = object_to_check.split(')')[0].split('(')[-1]
          return unit
        elif ', ' in object_to_check:
          unit = object_to_check.split(', ')[-1]
          return unit          
        else:                        
          return ''      


In [321]:
def special_table(df):
  '''this function handles sepcial tables: see Table 6 in An experimental approach to design self-consolidating concrete - ScienceDirect'''

  '''    **Parameters**

        df: *pandas Dataframe*
            A dataframe of the extracted table 
      
      **Returns**
      Updated dataFrame 
  '''

  df_headers = [header for header in df.columns if any(head in header for head in headers_to_look_for)]
  number_nan_rows = df.isnull().sum(axis=1).tolist()

  if number_nan_rows.count(len(df.columns) -1) > 1:
    pass
  else:
    return False, False, False
  count = 0
  list_of_appendages= []
  new_df = pd.DataFrame(columns = df.columns)
  for col in df_headers:
    appendage = ''
    for index, row in df.iterrows():
      final_row = [np.nan] * len(df.columns)
      if number_nan_rows[index-1] == len(df.columns)-1 and col == df.columns[0]: ###id must be in first index with all other column entries being Nan
        if row[col] != '': 
          appendage = '-'+row[col]
          count += 1 ##number of time auxillary row has an entry
        continue
      list_of_appendages.append(appendage)      
        
      for index2, final_col in enumerate(new_df.columns):
        if index2 == 0:
          final_row[index2] = row[final_col] + appendage
        else:
          final_row[index2] = row[final_col]
      row_series = pd.Series(final_row, index = new_df.columns)
      new_df = new_df.append(row_series, ignore_index=True)
  if count != 1 or list_of_appendages[0] == '':
    return new_df, True, False
  else: ## only 1 ancillary entry was found as in - Table 4 in article - The effect of zinc oxide additions on the performance of calcined sodium montmorillonite and illite
    first_col = list(new_df[new_df.columns[0]])
    updated_columns = ['UPDATED'] + list(new_df.columns[1:])
    new_df.columns = updated_columns
    new_col = []
    for col in first_col:
      new_col.append(col.split('-')[0])
    new_df['Code'] = new_col
    return new_df, True, True 

def remove_parenthesis(string):
  try:
    s1 = string.replace('(', '')
    s2 = s1.replace(')', '')
    return s2
  except:
    return string

def sandwhich_table(df, true_cols, current_col):
  '''See Table 2 in Changes in rheology and mechanical properties of ultra-high performance concrete with silica fume content - ScienceDirect'''
  header_num = []
  for i in true_cols:
    header_num.append(len(i))
  checker = header_num[1:-1]
  final_col = []
  for i, j in zip(true_cols, header_num):
    if j >1:
      for k in i[1]:
        unit = check_units(i[0][0])
        back = i[0][0].replace(unit, '')

        if unit == '':
          unit = check_units(k)
          k = k.replace(unit, '')

        back = remove_parenthesis(back)
        k = remove_parenthesis(k)
        
        if unit != '':
          final_col.append(back+'/' + k + ' ('+unit+')')
        else:
          final_col.append(back+'/' + k)
    else:
      unit = check_units(i[0])
      back = i[0].replace(unit, '')

      back = remove_parenthesis(back)
      
      if unit != '':
        final_col.append(back+'/' + ' ('+unit+')')
      else:
        final_col.append(back)      
      
  
  if any(len(col.split('/'))>2 for col in current_col):
    df.columns = final_col
    ##print('SANDWHICH', final_col)
    return df
  else:
    ##print('NO SANDWHCH')
    return df 

In [322]:
def should_table_be_transposed(df, thresh = 0.2):

    ''' this function determines whether the table extracted from the article should be transposed '''

    '''    **Parameters**

          df: *pandas DataFrame*
              df of the table that is checked.
          
        **Returns**
        True is the table should be transposed, False if it should be kept as is
    '''

    l_units = []

    keep = ['kg', '%', 'ratio', 'kg/m2', 'kg/m3', 'Pa', 'Pa.s', 'MPa', 'wt. %', 'h−1', 
            'min', '(min)', 'time', '°C', 'gravity']

    for unit in list_of_all_units:
      l_units.append('('+unit+')')
      l_units.append('('+unit)
      l_units.append(unit+')')
      if unit in keep:
        l_units.append(' '+unit)
      

    #list_of_all_units = l_units
    first_col = df.iloc[:, 0]
    count = 0


    must_transpose = [j for i in list(prop_dict.values()) for j in i]
        
    
    store = 0
    for col in df.columns[1:]:
      if any(header in df.columns[0] for header in headers_to_look_for) and '()' in col: #any(unit not in col for unit in list_of_all_units)):
        store+=1
    if store == len(df.columns)-1:
      return True

    if len(df.iloc[:, 0]) <2:
      return False
    


    for entry in first_col:
      if not any(header in df.columns[0] for header in headers_to_look_for) or any(the_col.split('(')[0] in must_transpose for the_col in df.columns):
        if any(units in str(entry) for units in l_units):
          count += 1
        elif any(comps in str(entry) for comps in compounds_to_look_for):
          count +=1
        else:
          continue

    if count >= thresh*len(list(first_col)):
      ##print('IT TRANSPOSED!!!!!')
      return True

    else:
      ##print('IT DID NOT TRANSPOSE!!!!!')
      return False

def omit_table(df, thresh = 0.5): 
  '''we omit this table if it contains multiple Nan values - meaning that the table contained blank entries that might correspond to any column 
  see - Table 3 in 'Changes in rheology and mechanical properties of ultra-high performance concrete with silica fume content '''
  '''if half of the tables have nan values greater than half of the entries for a column - then omit table from extraction'''
  number_nan = df.isna().sum()
  size_df = len(df)
  count = 0
  for i in number_nan:
    if i > size_df*thresh:
      count += 1
  if count > 1:
    return True
  else:
    return False

def make_first_colname_material(df):
    
  must_transpose = [j for i in list(prop_dict.values()) for j in i]
  if any(str(the_col).split('(')[0] in must_transpose for the_col in df[df.columns[0]]):
    df = df.rename(columns = {df.columns[0]: 'RAW CONSTITUENTS'})
  return df

def swap_first_and_second_column(df, thresh = 0.6):

    list_of_all_units = ['kg', '%', 'ratio', 'kg/m2', 'kg/m3', 'mm', 'Pa', 'Pa.s', 'g', 'MPa', 'wt. %', 'h−1', 'k']
    l_units = []

    for unit in list_of_all_units:
      l_units.append('('+unit+')')
      l_units.append('('+unit)

    list_of_all_units = l_units
    first_col = df.iloc[:, 0]
    second_col = df.iloc[:, 1]
    count1 = 0
    count2 = 0 

    for entry1, entry2 in zip(first_col, second_col):
      if any(units in str(entry1) for units in list_of_all_units):
        count1 += 1
      elif any(comps in str(entry1) for comps in compounds_to_look_for):
        count1 +=1

      if any(units in str(entry2) for units in list_of_all_units):
        count2 += 1
      elif any(comps in str(entry2) for comps in compounds_to_look_for):
        count2 +=1
      else:
        continue


    if count1 >= thresh*len(list(first_col)):
      return df
    elif count2 >= thresh*len(list(first_col)):
      df = df_column_switch(df, df.columns[0],df.columns[1])
      return df
    else:
      return df

def df_column_switch(df, column1, column2):
  i = list(df.columns)
  a, b = i.index(column1), i.index(column2)
  i[b], i[a] = i[a], i[b]
  df = df[i]
  return df

def transpose_table(df): 
  '''Takes in old table properties - title, dataframe, col_headers_true_false, and names and returns new props'''
  df = df.T.reset_index()
  new_header = df.iloc[0] 
  df = df[1:]
  h_list = []
  for header in list(new_header):
    if header == '':
      h_list.append('Code')
    else:
      h_list.append(header)

  df.columns = remove_math_symbols(h_list)

  new_titles = []
  for col in df.columns:
    if col == '':
      new_titles.append(['Code'])
    else:
      new_titles.append([col])

  return df, new_titles

In [323]:
'''Old approach'''
def old_approach(column_names, sub_column_names, sub_sub_column_names, n_columns):
    for i, name in enumerate(column_names): ## handling blank header i.e. headers are entries of each column
      if name == '':
        column_names[i] = 'BLANK HEADER'
    for i, name in enumerate(sub_column_names): ## handling blank header i.e. headers are entries of each column
      if name == '':
        sub_column_names[i] = 'BLANK HEADER'
    for i, name in enumerate(sub_sub_column_names): ## handling blank header i.e. headers are entries of each column
      if name == '':
        sub_sub_column_names[i] = 'BLANK HEADER'

    if n_columns < max([len(column_names), len(sub_column_names), len(sub_sub_column_names)]): ## case where first column entries are all headers - see Table 1 in ('Mixture design of concrete using simplex centroid design method - ScienceDirect.html'):
      first_column_is_header = True
      first_row_is_header = False
      if max([len(column_names), len(sub_column_names), len(sub_sub_column_names)]) == len(column_names):
          columns = column_names
          
      elif max([len(column_names), len(sub_column_names), len(sub_sub_column_names)]) == len(sub_column_names):
          columns = sub_column_names
      elif max([len(column_names), len(sub_column_names), len(sub_sub_column_names)]) == len(sub_sub_column_names):
          columns = sub_sub_column_names
    
    elif n_columns >  len(column_names) + len(sub_column_names) + len(sub_sub_column_names): ## handling cases where column headers are missing see Table 8 (Mixture design method of self-compacting lightweight aggregate concrete based on rheological property and strength of mortar - ScienceDirect.html)
          first_row_is_header = False
          first_column_is_header = True
      
    else: ##Other cases
      first_column_is_header = False
      first_row_is_header = False
      for i in range(n_columns):
        if n_columns == len(column_names) + len(sub_column_names) - i: ## if header and sub_header are perfectly split
            columns = column_names[0:len(column_names)-i]+sub_column_names            
            break
        elif n_columns == len(column_names) + len(sub_sub_column_names) - i: ## if header and second sub_header are perfectly split
            columns = column_names[0:len(column_names)-i]+sub_sub_column_names

            break
        elif n_columns == len(sub_column_names):
            columns = sub_column_names
            break
        elif n_columns == len(column_names):
            columns = column_names
            break 
    return columns                                       

In [324]:
## function to confirm if table header is actually a column of headers - see Table 1 in ('Mixture design of concrete using simplex centroid design method - ScienceDirect.html')
def column_checkers(df, thresh = 0.5): ##takes in a pandas dataframe
  breakers = [' ', '-', '%', '&', '–', '≥', '<', '±']
  known_col_header_names = ['No.']
  vacant_property_entries = ['–', None]
  column_is_header = []
  for col_index in range(len(df.columns)):
    if df.columns[col_index] in known_col_header_names:
      column_is_header.append(True)
      continue
    all_breakers_checker = 0#True
    for breaks in breakers:
      row_entry_is_property =0
      for i in range(len(df)):
        first_column_of_row = df.iloc[i][col_index]
        is_property = 0
        try:
          check_if_string = first_column_of_row.split(breaks) ## split by spaces
          for j in check_if_string:
            try:
              float(j)
            except:
              if j in vacant_property_entries:
                is_property +=0
              else:
                is_property += 1

        except:
          is_property = 0
        if is_property == 0: ## if all the entries converted to a float then the column is not a header
          continue
        else:
          row_entry_is_property +=1
    
      if row_entry_is_property >= len(df)*thresh: ## if the number of rows in the first column that are properties are greater than 80% of the entire number of columns the column is likely a header
        all_breakers_checker +=1  ## meaninng entire column is now classified as a property column 
      else:
        pass
    column_is_header.append(bool(all_breakers_checker > thresh * len(breakers)))
  return column_is_header


In [325]:
def remove_unit(string):
  '''this function removes strings containing units and returns a string without the unit and the unit itself'''
  list_of_all_units = ['°C','kg','kg/m', '%', 'ratio', 'kg/m2', 'kg/m3', 'mm', 'Pa', 'Pa.s', 'g', 'MPa', 'wt. %', 
                         'h−1', 'k', 'min', 'hours', 'time', 'set time', '°C', 'm2/kg', 'μm']
  list_of_all_units += ['ml/s']                       
  l_units = []

  
  keep = ['kg', '%', 'ratio', 'kg/m2', 'kg/m3', 'Pa', 'Pa.s', 'MPa', 'wt. %', 'h−1', 
          'min', '(min)', 'time', '°C', 'gravity']

  for unit in list_of_all_units:
    l_units.append('('+unit+')')
    #l_units.append('('+unit)
    #l_units.append(unit+')')
    #if unit in keep:
      # l_units.append(' '+unit)
  for unit in l_units:
    if unit in string:
      new_string = string.replace(unit, '')
      ##print(new_string)
      if '()' in new_string:
        new_string = new_string.replace('()', '')
      return new_string, unit
  if '()' in string:
    string = string.replace('()', '')
  return string, 'No unit'

def conversion_to_si(df_columns, unit):
  from pint import UnitRegistry
  ureg = UnitRegistry()
  new_column = []
  ##print(unit, df_columns)
  for i in df_columns:
      ##print(unit)
      try:
        value = i *ureg(unit)
        n_value = value.to_base_units().magnitude
      except:
        n_value = i
      try:
        new_column.append(float(n_value))
      except: ##cases where value is reported as a string
        try:
          new_column.append(float(n_value.split()[0]))
        except:
          new_column.append(0)
  ##print(new_column)
  return np.nanmean(new_column)


In [326]:
'''https://www.geeksforgeeks.org/python-code-#print-common-characters-two-strings-alphabetical-order/'''
from collections import Counter 
def common(str1,str2, get_length = False): 
      
    # convert both strings into counter dictionary 
    dict1 = Counter(str1) 
    dict2 = Counter(str2) 
    
    # take intersection of these dictionaries 
    commonDict = dict1 & dict2 
  
    if len(commonDict) == 0: 
        ##print (-1)
        return '0'
  
    # get a list of common elements 
    commonChars = list(commonDict.elements()) 
  
    # sort list in ascending order to #print resultant 
    # string on alphabetical order 
    commonChars = sorted(commonChars) 

    if get_length:
      return len(commonDict)

    # join characters without space to produce 
    # resultant string 
    return ''.join(commonChars)

In [327]:
def isNaN(num):
    try:
      if float('-inf') < float(num) < float('inf'):
          return False 
      else:
          return True
    except:
      return False

def remove_math_symbols_in_table(df):
  for i in df.columns:
    new_col = remove_math_symbols(list(df[i]))
    ##print('H', i, new_col)
    df[i] = new_col
  return df

def get_index_positions(list_of_elems, element):
    ''' Returns the indexes of all occurrences of give element in
    the list- listOfElements see https://thispointer.com/python-how-to-find-all-indexes-of-an-item-in-a-list/'''
    index_pos_list = []
    index_pos = 0
    while True:
        try:
            # Search for item in list from indexPos to the end of list
            index_pos = list_of_elems.index(element, index_pos)
            # Add the index position in list
            index_pos_list.append(index_pos)
            index_pos += 1
        except ValueError as e:
            break
    return index_pos_list

def pandas_should_have_this(df, tab_titles):
  '''this function joins rows that have the same mixture entries'''

  ## Here we correct the No! and No. discrepancy in the switch funcion in the make table if applicable
  no, no_ = [], []
  if 'No.' in df.columns and 'No!' in df.columns:
    for i in df['No.']:
      if not isNaN(i):
        try:
          new = str(float(i))
        except:
          new= str(i)
        no.append(new)
    for i in df['No!']:
      if not isNaN(i):
        try:
          new = str(float(i))
        except:
          new= str(i)        
        no_.append(new)
    no__ = no+no_
    ##print(len(no__), no__)
    df['No.'] = no__
    del df['No!']

  ##Here we clean up entries in table with math symbols
  
  df = remove_math_symbols_in_table(df)
  
  ##display(df)


  must_transpose = [j for i in list(prop_dict.values()) for j in i]  

  fin_mat_col = '/'.join(material.keys())

  df_headers = [header for header in df.columns if any(head in header for head in headers_to_look_for) and not any(compo in header for compo in must_transpose)]  
  new_df_cols = pd.Index(list(df.columns)+[fin_mat_col])
  new_df = pd.DataFrame(columns = new_df_cols)
  from collections import Counter
  checker = []
  ##print('Y', df_headers)
  for col in df_headers:    
    check = list(Counter(df[col]).values())
    ##print(check)
    ##print(df[col])#.values())
    checker.append(sum([1 for i in check if i > 1])) ##number of repeats in frame
  try:
    ##print(checker)
    col = df_headers[checker.index(max(checker))]
  except:
    pass
      
  used = []
  
  try:
    #print('THIS COL', col)
    pass
  except:
    return df
  table_id_numbers = [int(i.split('.')[0].split('Table')[1]) for i in tab_titles] #[for i in range(1, len(tab_titles)+1)]
  
  for index, row in df.iterrows():
    found = False
    which_tables = [tab_titles[index] for index, i in enumerate(table_id_numbers) if str(i) in row['Table Entry']]
    the_material = [mat for mat in [concrete_mortar_paste(t_table) for t_table in which_tables] if mat != None]
    try:
      if the_material.count(the_material[0]) == len(the_material):
        the_material = the_material[0]
      else:
        the_material = '/'.join(the_material)
    except:
      the_material = 'UNKNOWN' 
    for keys in material.keys():
      if any(mat in str(row[col]) for mat in material[keys]):
        the_material = concrete_mortar_paste(row[col])
        break
    for index2, row2 in df.iterrows():
      for col1 in df_headers:
        for col2 in df_headers:
          if row[col1] == row2[col2] and index != index2 and index2 not in used and index not in used:
            ###print('T', row[col1], row[col2])
            s1 = pd.Series(row.tolist())
            s2 = pd.Series(row2.tolist())
            new_row = s1.combine_first(s2)
            new_row = new_row.tolist()
            s1 = pd.Series(new_row+[the_material], index = new_df_cols)
            new_df = new_df.append(s1, ignore_index=True)
            used.append(index)
            used.append(index2)
            found = True
            break
    if not found and index not in used:
      used.append(index)
      hold = pd.Index(list(df.columns)+[fin_mat_col])      
      s1 = pd.Series(row.tolist()+[the_material], index = new_df_cols)#df.columns+['CONCRETE/MORTAR/PASTE'])
      new_df = new_df.append(s1, ignore_index=True)
  return new_df    

In [328]:
def get_compound_and_material(string):
  material, compound = 'NA', 'NA'
  for i in prop_dict.keys():
    if i in string:
      material = i
      break
  for i in compounds_to_look_for:
    if i in string:
      compound = i
      break    
  return material, compound

**Step 1 (Extract Article Tables
):** The first step involves of this process entails reading html files and parsing out tabular data from these files. In this step, the Beautifulsoup library was heavily used to read and extract data based on the pertinent html tags associated with table columns and rows in the file. Beautifulsoup uses various html parses but the htmlib4 was chosen. 

In [329]:
 def get_doi(path): ## returns paper doi
  '''this function extracts the doi of the article from its html file'''

  '''    **Parameters**

        path: *str*
            A string containing the path of the article to be extracted.

        **Returns**
          DOI of the article
  '''            
  soup = BeautifulSoup(path, 'html5lib')
  s = soup.find_all('html')
  h = s[0].find_all('head')
  for mname in h[0].find_all('meta'):
      mname = str(mname)
      if 'name="citation_doi"' in mname:
        doi = mname.split('content=')[1].split('name=')[0]
        doi = doi.replace('"', '')
        return doi    
 
 class HTMLTableParser:
       
        def parse_url(self, url):
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'xml') #'html5lib', 'html.parser', lxml, lxml-xml, xml 
            return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]

        def parse_html_file(self, path, table_id = 'All'):
            ''' This method extracts the tables from articles (i.e. path) and returns each table as a pandas dataFrame, along with its title and table headers'''

            '''    **Parameters**

                  path: *str*
                      A string containing the path of the article to be extracted.
                  table_id: *int* or 'All'
                      The specific ID of table to be extracted or extract 'All' the tables
                
                **Returns**
                A list of table properties (table title, pandas Dataframe, table headers)
            '''
            soup = BeautifulSoup(path, 'html5lib') ## lxml or html.parser or xml or html5lib
            #'html5lib', 'html.parser', lxml, lxml-xml, xml 

            all_tables = soup.find_all('table')
            all_divs = soup.find_all('div')
            table_title_index_list, table_title_list = [], []
            
            for i in all_divs:
              table_title = self.parse_html_table_title(i)
              if table_title != None:
                table_index = int(table_title.split('.')[0].split('Table')[1])

                if table_index not in table_title_index_list:
                  table_title_index_list.append(table_index)
                  table_title_list.append(table_title)
              else:
                continue
            
            c_m_p = concrete_mortar_paste(table_title_list)

            if table_id == 'All':
              all_list = []
              new_c_m_p = []
              for table, title, c_mp in zip(all_tables, table_title_list, c_m_p):
                try:
                  all_list.append(self.parse_html_table(table, title, c_mp))                 
                except:
                  pass
              return all_list
            else:  
              return self.parse_html_table(all_tables[table_id-1], table_title_list[table_id-1], c_m_p[table_id-1])


        def parse_html_table_title(self, div): ## returns table name and title
          
          '''this method extracts the title of a table'''
          
          if div.find_all('table') != []:

            if div.find_all('span') != []:

              for spans in div.find_all('span'):

                spans = str(spans)
                if 'span class="label"' in spans:
                  spans_list = spans.split('span class="label"')[1].split('</span>') ## this identifies the Table id
                  table_id = spans_list[0]
                                  
                  
                  if 'Table' in table_id:
                    table_title = spans_list[1]
                    table_title = BeautifulSoup(table_title)
                    table_title = table_title.get_text()

                    table_title = table_id+table_title
                    table_title = table_title.replace('>', '')
                    
                    return table_title
                   
          return None

        def parse_html_table(self, table, title, c_m_p):
            
            '''this method returns the table as pandas DataFrame and table headers'''
            n_columns = 0
            n_rows=0
            column_names = []
            sub_column_names= []
            sub_sub_column_names = []
            header_row_count = 0
            first_non_header_row = [] 
            move_to_second_column = False

            # Find number of rows and columns, we also find the column titles if we can

            for row in table.find_all('tr'):
                
                # Determine the number of rows in the table
                td_tags = row.find_all('td')

                holder = 0 

                if len(td_tags) >= holder: ### switched from 0 to 1 - assumes that every table has at least two columns
                    n_rows+=1
                    if n_columns == 0:
                        # Set the number of columns for our table
                        n_columns = len(td_tags)
                        for the_column in td_tags: ## getting the first non header row
                          first_non_header_row.append(the_column.get_text())                          
                          move_to_second_column = True


                        
                # Handle column names if we find them
                th_tags = row.find_all('th') 
                if len(th_tags) > 0 and len(column_names) == 0 and header_row_count == 0:
                    for th in th_tags:
                        column_names.append(th.get_text())                                              
                    header_row_count += 1
                
                # Handle sub_column_names if we find them
                elif len(th_tags) > 0 and len(sub_column_names) == 0 and header_row_count == 1:
                    for th in th_tags:
                      sub_column_names.append(th.get_text())
                    header_row_count += 1                
                
                #Handles the second sub header if we find them
                elif len(th_tags) > 0 and len(sub_sub_column_names) == 0 and header_row_count == 2: ## Cases where there are two sub headers
                    for th in th_tags:
                      sub_sub_column_names.append(th.get_text())                
                

            header_row_count = 0
            colspan_column_names = []
            colspan_sub_column_names = []
            colspan_sub_sub_column_names = []

            for row in table.find_all('tr'):
                th_tags = row.find_all('th')
                if header_row_count == 0: 
                  for th in th_tags:
                    if 'colspan' in th.attrs and th.attrs['colspan'].isdigit():
                      col = int(th.attrs['colspan'])
                      colspan_column_names.append(col)
                    else:
                      colspan_column_names.append(1)
                  header_row_count += 1

                elif header_row_count == 1: 
                  for th in th_tags:
                    if 'colspan' in th.attrs and th.attrs['colspan'].isdigit():

                      col = int(th.attrs['colspan'])
                      colspan_column_names.append(col)
                    else:
                      colspan_sub_column_names.append(1)                    
                  header_row_count += 1

                elif header_row_count == 2: 
                  for th in th_tags:

                    if 'colspan' in th.attrs and th.attrs['colspan'].isdigit():
                      col = int(th.attrs['colspan'])
                      colspan_column_names.append(col)
                    else:
                      colspan_sub_sub_column_names.append(1)
                  header_row_count += 1                  
                    

            def get_rid_of_empty_space(col_name, colspan):
              new_col_name = []
              new_colspan = []
              
              for i, j in zip(col_name, colspan):
                if i != '':
                  new_col_name.append(i)
                  new_colspan.append(j)
              return new_col_name, new_colspan

            def update_empty_space_in_first_row(col_name, colspan):
              new_col_name = []
              new_colspan = []
              use_old_approach = False
              empyt_first_column = False
              index_track = -1
              for i, j in zip(col_name, colspan):
                index_track+=1
                
                if i != '':
                  new_col_name.append(i)
                  new_colspan.append(j)
                
                else:  
                  if index_track == 0:
                    new_col_name.append('Mix / COMP')
                    new_colspan.append(1)
                    empyt_first_column = True
                    use_old_approach = True
                  
                  else:
                    if new_colspan[index_track-1] == 0:
                      new_colspan[index_track-1] += 1
                    else:
                      new_col_name.append('BLANK HEADER')
                      new_colspan.append(j)
                                 
              return new_col_name, new_colspan, use_old_approach
            
            column_names, colspan_column_names, use_old_approach = update_empty_space_in_first_row(column_names, colspan_column_names)            
            
            sub_column_names, colspan_sub_column_names = get_rid_of_empty_space(sub_column_names, colspan_sub_column_names)
            
            sub_sub_column_names, colspan_sub_sub_column_names = get_rid_of_empty_space(sub_sub_column_names, colspan_sub_sub_column_names)

            ## Using method 1 to determine column names
            if not use_old_approach and max(len(colspan_column_names), len(colspan_sub_column_names), len(colspan_sub_sub_column_names)) != 0: 
              columns = []
              cols_col_index_rec = 0
              sub_cols_index_rec = 0
              sub_sub_cols_index_rec = 0
              full_header_list = []
              for col, span in zip(column_names, colspan_column_names):

                if span == 1:
                  columns.append(col)
                  full_header_list.append([col])

                else:
                  holder_list= [[col]]
                  holder_list.append(sub_column_names[sub_cols_index_rec:span+sub_cols_index_rec])
                                   
                  for sub_index, sub_span in enumerate(colspan_sub_column_names[sub_cols_index_rec:span+sub_cols_index_rec]):

                    if sub_span == 1:
                      columns.append(sub_column_names[sub_cols_index_rec+sub_index])

                    else:
                      holder_list.append(sub_column_names[sub_sub_cols_index_rec:sub_span+sub_sub_cols_index_rec])
                      for sub_sub_index, sub_sub_span in enumerate(colspan_sub_sub_column_names[sub_sub_cols_index_rec:sub_span+sub_sub_cols_index_rec]):               
                        columns.append(sub_sub_column_names[sub_sub_cols_index_rec+sub_sub_index])

                  full_header_list.append(holder_list)
                  sub_cols_index_rec += span
                  sub_sub_cols_index_rec +=  sub_span
            
            ## Using method 2 (an older approach) to determine column names
            else:
              columns = old_approach(column_names, sub_column_names, sub_sub_column_names, n_columns)
              cols_col_index_rec = 0
              sub_cols_index_rec = 0
              sub_sub_cols_index_rec = 0
              full_header_list = []
              for col, span in zip(column_names, colspan_column_names):
                if span == 1:
                  full_header_list.append([col])
                else:
                  holder_list= [[col]]
                  holder_list.append(sub_column_names[sub_cols_index_rec:span+sub_cols_index_rec])
                                   
                  for sub_index, sub_span in enumerate(colspan_sub_column_names[sub_cols_index_rec:span+sub_cols_index_rec]):

                    if sub_span == 1:
                      pass
                  
                    else:
                      holder_list.append(sub_column_names[sub_sub_cols_index_rec:sub_span+sub_sub_cols_index_rec])
                    sub_sub_cols_index_rec +=  sub_span
                  full_header_list.append(holder_list)
                  sub_cols_index_rec += span
                  

            def priority_headers(columns):
              #switch_2 = 1
              change_No = False
              mix_headers =  [header for header in columns if any(head in header for head in headers_to_look_for) and not any('('+unit_+')' in header for unit_ in list_of_all_units)] ##gets all Ids
              
              new_columns = []
              ##print('O')
              for index, col in enumerate(columns):
                if col in mix_headers:
                  if mix_headers.count(col) > 1:
                    new_columns.append('!'*index+str(col))
                  else:
                    new_columns.append(col)
                else:
                  new_columns.append(col)
              #print('R', new_columns, columns)  
              #columns = new_columns  
              for index, col in enumerate(columns):
                if 'No.' in columns and 'No.' not in col and any(header in col for header in headers_to_look_for):
                  switch_2 = index
                  change_No = True


          
              if 'No.' in columns and change_No: ##No and material/mix etc
                switch_1 = columns.index('No.')
                columns[switch_1] = 'No!'
                ###print(columns)
                return columns, switch_1, switch_2

              else:
                return columns, 1, 1

            columns, switch_1, switch_2 = priority_headers(columns)
            
            ###print('SW', switch_1, switch_2)
            


            columns = remove_math_symbols(columns)
            final_column_names = columns
            df = pd.DataFrame(columns = columns,
                              index= range(0,n_rows))
            
            #row_marker = 0
 

            '''Here we build the tables'''
 
            final_row_list = []
            final_final_row_list = []
            max_copy_rows = -1
            
            for row_marker, row in enumerate(table.find_all('tr')):
                empty = True
                column_marker = 0

                if row_marker == 0:
                  pass
                
                columns = row.find_all('td')
                if len(columns) == len(final_column_names)-1:
                  columns=row.find_all(['th', 'td'])


                holder_row_list = []
               
                
                if row_marker > max_copy_rows:
                  final_final_row_list = []
                  for column in columns:
                      final_row_list = []
                      if 'rowspan' in column.attrs and column.attrs['rowspan'].isdigit():
                        number_row = int(column.attrs['rowspan'])                                              
                        holder_row_list = [row_marker+ i for i in range(1, number_row)]

                        for i in holder_row_list:
                          final_row_list.append(i)
                        
                        final_final_row_list.append(final_row_list)
                        max_copy_rows = max(final_row_list)

                    
                      else:
                        final_final_row_list.append([])
                        pass


                confirm = True
                for index, i in enumerate(final_final_row_list):
                  if i == []:
                    col_index_to_add = index 
                    confirm = False
                    break
                                
                try:
                  new_add_rows_name +=1
                except:
                  pass
                
                for col_index, column in enumerate(columns):

                    if row_marker in final_final_row_list[col_index]:                      
                      add_rows_name +=1                      
                      df.iat[row_marker,column_marker] = copy_forward_list[col_index]+'-'+str(new_add_rows_name) #copy_forward+'-'+str(add_rows_name)                                             
                      df.iat[row_marker,col_index+col_index_to_add] = column.get_text()

                    elif final_final_row_list[col_index] == [] and len(columns) < n_columns:
                       df.iat[row_marker,column_marker+col_index_to_add] = column.get_text()
                    else:
                      df.iat[row_marker,column_marker] = column.get_text()                      
                      
                      if column_marker == 0:
                        copy_forward = column.get_text()
                        add_rows_name = 0
                        new_add_rows_name = 0
                        copy_forward_list = []                        
                      copy_forward_list.append(column.get_text())

                    column_marker += 1

            # Convert to float if possible
            for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass

            df.dropna(0, 'all', inplace=True)

            if switch_1 == 0:
              df = df_column_switch(df, df.columns[switch_1], df.columns[switch_2])
              final_column_names = list(df.columns)

            
            column_is_header = column_checkers(df)
            is_first_column_a_header = column_is_header[0]
            
            
            ### this section fills the column and sub_column_names and determines units

            if  'BLANK HEADER' not in df.columns:
              pass 

            else:
              if is_first_column_a_header and len(sub_column_names) > len(sub_sub_column_names):
                if sub_column_names[0] == 'BLANK HEADER':
                  final_column_names = [column_names[0]]+sub_column_names[1:]                  
                else:  
                  final_column_names = [column_names[0]]+sub_column_names
                df.columns = final_column_names
              elif is_first_column_a_header and len(sub_column_names) < len(sub_sub_column_names):
                final_column_names = [column_names[0]]+sub_sub_column_names
                df.columns = final_column_names
            
            if len(sub_column_names) >= len(sub_sub_column_names):
              likely_unit_col = column_names
              likely_colspan = colspan_column_names
            elif len(sub_column_names) < len(sub_sub_column_names):
              likely_unit_col = sub_column_names
              likely_colspan = colspan_sub_column_names



            number_of_same_entries = sum(x == y for x, y in zip(final_column_names, likely_unit_col))
            if number_of_same_entries != 0 and number_of_same_entries != len(final_column_names):
              if check_units(likely_unit_col).count('No unit') == len(likely_unit_col): ##checking if units are in title
                the_unit = check_units(title)
                omit_unit = ['No.', 'Mix'] ## names of column that generally dont have units
                for index, col in enumerate(final_column_names):
                  if not column_is_header[index] and not check_units(col, True):
                    final_column_names[index] = col +'('+the_unit+')'
                    df.columns = final_column_names
              else:
                the_unit = check_units(likely_unit_col)
                for index_p, i in enumerate(range(number_of_same_entries, len(likely_unit_col))):
                  from_ = likely_colspan[i-1]*index_p+number_of_same_entries
                  try:
                    if the_unit[i+1] == 'No unit':
                      to_ = from_+ likely_colspan[i]+1
                    else:
                      to_ = from_+ likely_colspan[i]

                  except:
                      to_ = from_+ likely_colspan[i]
                  
                  for index, col in zip(range(from_,to_), final_column_names[from_:to_]):
                    if not check_units(col, True):
                      final_column_names[index] = likely_unit_col[i]+'/'+ col +'('+the_unit[i]+')'
                    else:
                      final_column_names[index] = likely_unit_col[i]+'/'+ col
                df.columns = final_column_names             
            
            else:
              if check_units(likely_unit_col).count('No unit') == len(likely_unit_col): ##checking if units are in title
                the_unit = check_units(title)
                omit_unit = ['No.', 'Mix'] ## names of column that generally dont have units
                for index, col in enumerate(final_column_names):
                  if not column_is_header[index] and not check_units(col, True):
                    final_column_names[index] = col +'('+the_unit+')'
                    df.columns = final_column_names
              else:
                pass
            


            df = swap_first_and_second_column(df)

            # ##print('1')
            # #display(df)

            df = convert_ids_to_string(df)

            # ##print('2')
            # #display(df)


            if should_table_be_transposed(df):              
              new_df, new_full_header_list = transpose_table(df)
              new_df = make_first_colname_material(new_df)
              new_df.columns = clean_table_headers(new_df.columns)
              return [title, new_df, column_is_header, new_full_header_list, c_m_p]

            df.columns = clean_table_headers(df.columns)
            pot_new_df, t_f, new_col_added = special_table(df) 

            # ##print('3')
            # #display(df)

            if t_f:
              df = pot_new_df.copy(deep = True)
            if new_col_added:
              column_is_header.append(True)
              full_header_list.append([df.columns[-1]])
            df = make_first_colname_material(df)

            # ##print('4')
            # #display(df)

            if omit_table(df):
              #display(df)
              raise Exception('This table has too many empty cells so it wasnt extracted')

            
            df = sandwhich_table(df, full_header_list, df.columns)
            
            # ##print('5')
            # #display(df)            
            
            ###print(title, df, column_is_header, full_header_list, c_m_p)
            
            return [title, df, column_is_header, full_header_list, c_m_p]


**Step 2 (Link all article Tables by mixture codes):** This process links each table in an article by their mixture identification (e.g. Mix no.) and chemical compositions of raw materials (e.g. OPC and FAM). This results in a aggregated table for each article containing its studied mixtures, their associated material and compositional properties, article DOI, and the article title.

In [330]:
class ExtractDF:

    '''takes in a list of tuples - table properties i.e. title, df, column_rows_are_headers, list of headers and sub_headers, and whether to transpose the table or not
     for each paper and links the tables together'''

    def __init__(self, t_prop, article_title, article_doi):
      self.t_prop = t_prop[0]#[0] #'''all tables are linked together'''
      self.article_title = article_title
      self.doi = article_doi      


    def check_improper_table_heading(self):
      '''this method updates the tabular headers for tables in a particular articles'''

      
      df_of_tables = [table[1] for table in self.t_prop]
      df_headers = [[header for header in df.columns if any(head in header for head in headers_to_look_for) and not any('('+unit_+')' in header for unit_ in list_of_all_units)] for df in df_of_tables]
      df_title = [table[0] for table in self.t_prop]

      def update_id_with_title(title):
        broken = title.split()
        ###print(broken)
        for index, breaks in enumerate(broken):
          if any(breaks == unit for unit in list_of_all_units):
            return ' '.join(broken[index-1:])
          elif '%' in breaks:
            return ' '.join(broken[index:])

        try:
          ###print('A')
          return ' '.join(broken[:5]) ##first 5 entries
        except:
          ###print('Y')
          return ' '.join(broken[:])
              

      update_headers = []
      checker_list = []
      for df1, headers1 in zip(df_of_tables, df_headers):
        df1_cols = df1.columns
        for df2, headers2 in zip(df_of_tables, df_headers):
          df2_cols = df2.columns
          if not df1.equals(df2):
            for head1 in headers1:
              df1_entry = list(df1[head1])
              for head2 in headers2:
                df2_entry = list(df2[head2])
                similar_entry = len(set(df1_entry) & set(df2_entry))
                if similar_entry > 0 and head2 != head1:
                  new_entry = '/'.join(sorted([head1, head2]))
                  old_entry = head1
                  if list(df1.columns) not in checker_list: 
                    update_headers.append([df1, {old_entry: new_entry}])
                    checker_list.append(list(df1.columns))
      new_df = []
      for df in df_of_tables:
        got_it = False
        for df_to_up in update_headers:
          if df_to_up[0].equals(df):
           df.rename(columns = df_to_up[1], inplace = True)
           new_df.append(df)
           got_it = True
        if not got_it:
          new_df.append(df)
        
      
      new_df_mix_headers = [[header for header in df.columns if any(head in header for head in headers_to_look_for) and not any('('+unit_+')' in header for unit_ in list_of_all_units)] for df in new_df]
      '''this method looks for tables that repeat the same columns and first column IDs - see '''
      #CASE A same headers and first columns across each table
      dict_ = {}
      joined_col = []
      for n_df, title in zip(new_df, df_title):
        columns = ''.join(list(n_df.columns))
        mix_ids = ''.join(map(str, list(n_df.iloc[:, 0]))) ##first col only 
        table_id = int(title.split('.')[0].split('Table')[-1])
        joined_col.append([columns, mix_ids, table_id])
      
      tables_to_check = []
      for col1 in joined_col:
        for col2 in joined_col:
          if col1[2] != col2[2] and len(common(col1[0], col2[0])) >= 0.9*max(len(col1[0]), len(col2[0])):
            if len(common(col1[1], col2[1])) >= 0.9*max(len(col1[1]), len(col2[1])):
              tables_to_check.append(col1[2])
              tables_to_check.append(col2[2])

      tables_to_check = list(set(tables_to_check))


      for n_df, headers, title in zip(new_df, new_df_mix_headers, df_title):
        if any('Table '+str(id) in title for id in tables_to_check):
          new_mixture_ids = []

          for mix in list(n_df.iloc[:, 0]):
            #got_it = False
            ###print('A', holder_rep
            ###print(str(mix) + '-' + update_id_with_title(''.join(title.split('.')[1])))
            try:
              new_mix = str(mix) + '-' + update_id_with_title(''.join(title.split('.')[1]))
            except:
              new_mix = mix
            new_mixture_ids.append(new_mix)          
          n_df.iloc[:, 0] = new_mixture_ids
        ##display(n_df)
      #CASE B Same mixture ID-s within the table 
      for n_df, headers, title in zip(new_df, new_df_mix_headers, df_title):
        columns = n_df.columns
        repeat = [] 
        for index1, header in enumerate(headers):
          index_of_header = get_index_positions(list(columns), header)
          if len(index_of_header) == 1:
            index_of_header = index_of_header[0]
            mixture_entries = list(n_df[header])
          else:  
            repeat.append(header)
            break

          holder_new = []
          holder_rep = []
          for mixture in mixture_entries:
            if mixture not in holder_new:
              holder_new.append(mixture)
            else:
              holder_rep.append(mixture)
          new_mixture_ids = []
          for index, mix in enumerate(mixture_entries):

            if mix in control_ids:
              new_mix = mix + '-' + update_id_with_title(''.join(title.split('.')[1]))
              new_mixture_ids.append(new_mix) 
             
            elif mix in holder_rep and not isNaN(mix):
              try:
                new_mix = mix+'-'+columns[index_of_header+1]+'='+str(n_df.iloc[index, index_of_header+1])
              except:
                new_mix = mix+'-'+columns[index_of_header+-1]+'='+str(n_df.iloc[index, index_of_header-1])

              new_mixture_ids.append(new_mix)
            else:
               new_mixture_ids.append(mix)

          n_df.iloc[:, index_of_header] = new_mixture_ids
          ##display(n_df)

      for i, n_df in zip(list(range(len(self.t_prop))), new_df):
        self.t_prop[i][1] = n_df
      
      return self.t_prop
        

    def extract_compositions(self):
      '''this method extracts the composition of cement, fly ash and slag in the article'''

      keep = []
      n_compounds_to_look_for = compounds_to_look_for + keep  #+ list_of_all_units                            
      combinations = [' ', '', '(', ' (']
      new_comp = []
      for comp in n_compounds_to_look_for:
        n_l = [comb1+comp+comb2 for comb1 in combinations for comb2 in combinations]
        new_comp = new_comp + n_l

      n_compounds_to_look_for = n_compounds_to_look_for + new_comp
      full_header_list = []
      full_property_list = []
      for table_id, table in enumerate(self.t_prop):
        df = table[1]
        table_columns = df.columns
        first_col = table_columns[0]

        for index, row in df.iterrows():
          holder_list = []
          header_list = []
          for material in prop_dict.keys():
            if any(prop in str(row[first_col]) for prop in prop_dict[material]): ##if material is in table 
             for cols in table_columns:
               if any(comps in str(cols) for comps in n_compounds_to_look_for): ##composition is in row
                full_property_list.append(row[cols])
                if '(' in row[first_col] and '%' not in row[first_col]:
                  full_header_list.append(cols +' in/for '+row[first_col].split('(')[0] + ' ('+material+')' )
                else:
                  full_header_list.append(cols +' in/for '+row[first_col] + ' ('+material+')' )
      final_header = []

      holder = []
      holder_prop = []
      for i, j in zip(full_header_list, full_property_list):
        if j != '–' and j != np.nan:
          material, compound = get_compound_and_material(i)
          holder.append(compound +' in/for '+material)
          holder_prop.append(j) 
      full_header_list = holder
      full_property_list = holder_prop

      for i in full_header_list:
        to_add = set(i)
        for j in to_add:
          if j not in final_header:
            final_header.append(j)

      final_header = full_header_list
      new_df = pd.DataFrame(columns = final_header)
      final_row = [np.nan]*len(final_header)
      for prop, prop_header in zip(full_property_list, full_header_list):
          index_of_val = final_header.index(prop_header)
          final_row[index_of_val] = prop
      row_series = pd.Series(final_row, index = new_df.columns)

      number_nan = row_series.isna().sum(axis = 0)
      number_nan = final_row.count(np.nan)
      new_df = new_df.append(row_series, ignore_index=True)
      new_df = new_df.dropna(thresh=3)
      new_df.reset_index(drop=True, inplace=True)

      return new_df

    #def mixture_checker(columns):


    def make_table(self):
      ''''extracting propery data based on mixture identification'''

      all_id_w_table = []
      
      # headers_to_look_for = ['code', 'ID', 'No.', 'Mix', 'Mixtures', 'Code', 'name', 'mix', 'Mixture', 'Notation']
      
      table_id_list = []
      table_title_list= []
      splitters = [' ', '/', '.']

      for table_id, table in enumerate(self.t_prop):
        table_id_list.append(int(table[0].split('.')[0].split('Table')[1]))
        ids = []
        table_title_list.append(table[0])
        df_columns = table[1].columns
        ##print(table[2], df_columns)
        for splitter in splitters:
          for index, col_header in enumerate(table[2]):
              if col_header and any(item in df_columns[index].split(splitter) for item in headers_to_look_for) and not any('('+unit_+')' in df_columns[index].split(splitter) for unit_ in list_of_all_units): ## conditionals for 
                ###print(df_columns[index])
                if list(df_columns).count(df_columns[index]) == 1:
                  loop_around = table[1][df_columns[index]]
                  
                else:
                  loop_around = table[1].iloc[:, index]
                for entries in loop_around:
                  if entries not in ids:
                    ids.append(entries)
        if ids != []:
          t_id = int(table[0].split('.')[0].split('Table')[1])
          all_id_w_table.append([ids, t_id])      

      '''searching for which ids repeat across different tables'''
      removable_characters = ['-', '–']
      non_entry_characters = ['–', '–']
      
      for remove in non_entry_characters:
        for index, i in enumerate(all_id_w_table):
          try:
            all_id_w_table[index][0].remove(remove)
          except:
            pass

      ignore_sims = [np.nan, '', 'BLANK', 'RAW CONSTITUENTS']
      new_id_list_w_table = []
      for remove in removable_characters:
        for index, ids_in_table in enumerate(all_id_w_table):
          compare_list = list(range(len(all_id_w_table)))
          compare_list.remove(index)
          ids_to_keep = []
          for checking_index in compare_list:
              for id1 in ids_in_table[0]:
                 for id2 in all_id_w_table[checking_index][0]:
                   checker = min(len(str(id1).replace(remove, '')), len(str(id2).replace(remove, '')))
                   if len(common(str(id1), str(id2))) >= checker:
                      ids_to_keep.append(id1)
                   else:
                      ids_to_keep.append(id1)        ### DELETE THIS CODE BLOCK IF EDITING                                 
          holder_list = []
          ids_to_keep = list(dict.fromkeys(ids_to_keep))
          for i in ids_to_keep:
            if i not in ignore_sims:
              holder_list.append(i)
          if holder_list != []:
             new_id_list_w_table.append([holder_list, ids_in_table[1]])
      dictm = {}
      
      if new_id_list_w_table == []:
        new_id_list_w_table = all_id_w_table            

      for index, table in enumerate(new_id_list_w_table):
        holder = [table[1]]
        for index2, table2 in enumerate(new_id_list_w_table):

          if index != index2 or len(new_id_list_w_table) == 1:
            
            for entry in table[0]:
              for entry2 in table2[0]:
                if entry == entry2:
                  if table2[1] not in holder:
                    holder.append(table2[1])
                dictm[entry] = holder

      rows = []
      rows_headers = []
      headers = []
      ###print(dictm.keys())
      for key, value in zip(dictm.keys(), dictm.values()):        
        for table in value:          
          df = self.t_prop[table_id_list.index(table)][1]
          for index, cols in enumerate(df.columns):
            if list(df.columns).count(df.columns[index]) == 1: ## if col_name not repated in table
                list_row = df.loc[df[cols] == key].values.flatten().tolist()
                if len(df.columns) != len(list_row):
                  number_rep = list_row.count(key)
                  for i in range(number_rep):
                    m_holder_list = list_row[i*len(df.columns):(i+1)*len(df.columns)]
                    ###print(m_holder_list)
                    ###print(key)
                    index_of_key = m_holder_list.index(key)
                    try: 
                      m_holder_list[index_of_key] = str(key)+'-'+str(df.columns[0])+'='+str(m_holder_list[0])
                    except: 
                      m_holder_list[index_of_key] = str(key)+'-'+str(df.columns[index_of_key + 1])+'='+str(m_holder_list[index_of_key + 1])
                    value.sort()
                    
                    str_value = ['['+str(v)+']' for v in value]
                    table_number = ''.join(str_value)
                         
                    rows.append(m_holder_list+[table_number])
                    rows_headers.append(df.columns.tolist()+['Table Entry'])
                  list_row = []
            else:
                mask = df.iloc[:, index] == key
                pos = np.flatnonzero(mask)
                list_row = df.iloc[pos].values.flatten().tolist()
            try:
              if list_row != []:
                value.sort()
                
                str_value = ['['+str(v)+']' for v in value]
                table_number = ''.join(str_value)
                                
                rows.append(list_row+[table_number])
                rows_headers.append(df.columns.tolist()+['Table Entry'])
                for i in df.columns.tolist():
                  if i not in headers:
                    headers.append(i)
              else:
                continue
            except:
              continue
      
      ### spliting up rows with different IDS in each row
      new_rows = []
      new_rows_header = []
      top_row_index = -1
      row_index = -1 
      for row, row_header in zip(rows, rows_headers):
            top_row_index += 1
            row_index = -1 
            for row2, row_header2 in zip(rows, rows_headers):
              row_index += 1 
              for index_j, j in enumerate(row2):
                if row[0] == j and row != row2 and (index_j != 0):
                  for index_i, i in enumerate(row):
                    if row_header2[index_j] == row_header[index_i]:# found the id that is present eslewhere or row_header2 == row_header:
                      if row_header2.count(row_header2[index_j]) > 1: ## if header is repeated  in the row
                        new_row_to_add = row2[index_j:]
                        new_row_header_to_add = row_header2[index_j:]
                        rows[row_index][index_j:] = '-'
                        rows_headers[row_index][index_j:] = '-'                      
                        if new_row_to_add not in new_rows:
                          new_rows.append(new_row_to_add)
                          new_rows_header.append(new_row_header_to_add)

      rows = rows+new_rows
      rows_headers = rows_headers + new_rows_header
      

      ## cleaning 0 - removing rows with different dash or blank characters 
      charac_to_remove = ['', '-', '–', '–', '–']
      new_rows = []
      new_rows_headers = []
      index_a = -1
      
      for row, header in zip(rows, rows_headers):
        index_a +=1
        if len(row) != len(header):
          continue
        holder_list, holder_header = [], []
        for index, entry in enumerate(row):
           if entry not in charac_to_remove:
             holder_list.append(entry)
             holder_header.append(header[index])
        new_rows.append(holder_list)
        new_rows_headers.append(holder_header)

      rows = new_rows
      rows_headers = new_rows_headers

      

      ## cleaning 1 - removing any repeated entries
      new_rows = []
      new_rows_headers = []
      indexz = -1
      
      for row, header in zip(rows, rows_headers):
        indexz += 1
        new_row_entry = []
        new_header_entry = []
        for index, entry in enumerate(row):
          if entry not in charac_to_remove:
            new_row_entry.append(entry)
            new_header_entry.append(header[index])

        if new_row_entry not in new_rows:
          new_rows.append(new_row_entry)
          new_rows_headers.append(new_header_entry)
        else:
          continue
      rows = new_rows
      rows_headers = new_rows_headers

      # cleaning 2 - joining all rows with identical first entry i.e. ID
      used = []
      new_rows = []
      new_rows_header = []
      for row, row_header in zip(rows, rows_headers):
          holder_row = row
          holder_row_header = row_header
          if row[0] not in used:
            for row2, row_header2 in zip(rows, rows_headers):
              if row != row2 and row[0] not in used:
                if row[0] == row2[0]:
                  holder_row = holder_row + row2[1:]
                  holder_row_header = holder_row_header + row_header2[1:]
            used.append(row[0])
            new_rows.append(holder_row)
            new_rows_header.append(holder_row_header)
      rows = new_rows
      rows_headers = new_rows_header

      
      import itertools
      headers = []
      for row in rows_headers:
        for r in row:
          if r not in headers: 
            headers.append(r)
      
      
      new_df = pd.DataFrame(columns = headers)
      for row, row_header in zip(rows, rows_headers):
          final_row = [np.nan]*len(headers)
          for r, r_h in zip(row, row_header):
            index_of_val = headers.index(r_h)
            final_row[index_of_val] = r
          row_series = pd.Series(final_row, index = new_df.columns)
          new_df = new_df.append(row_series, ignore_index=True)
      new_df = new_df.dropna(thresh=2)
      new_df.reset_index(drop=True, inplace=True)
      new_df['Article Title'] = len(new_df)*[self.article_title]
      new_df['DOI'] = len(new_df)*[self.doi]

      ### last step is grouping rows wit identical IDs###
      to_group = []
      non_group = []
      for col in new_df.columns:
        if any(ids in col for ids in headers_to_look_for):
          to_group.append(col)
        else:
          non_group.append(col)
      final_new_df = new_df
      self.new_df = final_new_df
      return final_new_df, table_title_list


def join_comp_with_mix(mixtures, compositions):
  if len(mixtures) != 0:
    new_comp  = pd.DataFrame(np.repeat(compositions.values,len(mixtures),axis=0))
    new_comp.columns  = compositions.columns
    result = pd.concat([mixtures, new_comp], axis = 1)
    return result
  else:
    return compositions

def prettify(df):

    df.columns = clean_table_headers(df.columns)
    
    comp_headers_to_look_for = [j for i in list(prop_dict.values()) for j in i]
    

    df_mix_headers = [header for header in df.columns if any(head in header for head in headers_to_look_for) and not any(head in header for head in comp_headers_to_look_for) and not any('('+unit_+')' in header for unit_ in list_of_all_units)]
    ###print(df_mix_headers)
    ###print(headers_to_look_for)
    df_comp_headers = [header for header in df.columns if any(head in header for head in comp_headers_to_look_for)]

    df_comp_headers = [header for header in df.columns if any(head in header for head in compounds_to_look_for)]
    
    headers_to_avoid = df_mix_headers+df_comp_headers+['Article Title', 'CONCRETE/MORTAR/PASTE', 'DOI']

    ##print('A', df_mix_headers)
    ##display(df)
    for index, header in enumerate(df_mix_headers): #renaming all mixtures columns
      new_entry = []
      ###print(df[header])
      for i in df[header]:
          if '()' in str(i):
            i = i.replace('()', '')
            new_entry.append(i)
          else:
            new_entry.append(i)
      ##print('B', new_entry)
      df[header] = new_entry
      df = df.rename(columns = {header: 'CODE/MIX TAG-'+str(index+1)})
      headers_to_avoid.append('CODE/MIX TAG-'+str(index+1))

    for index, header in enumerate(df_comp_headers):
      df = df.rename(columns = {header: 'COMPOSITION: '+header})
      headers_to_avoid.append('COMPOSITION: '+header)


#    '''MIGHT NEED - DONT REMOVE!!!!!!
    for header in df.columns:
      if header not in headers_to_avoid:
        df  = df.rename(columns = {header: 'PROPERTY: '+header})
        #'''    

    df.sort_index(axis=1, inplace=True)

    return df

    if any(the_col.split('(')[0] in must_transpose for the_col in df[df.columns[0]]):
      bs = [the_col.split('(')[0] in must_transpose for the_col in df[df.columns[0]]]
      df = df.rename(columns = {df.columns[0]: 'RAW CONSTITUENTS'})


**Step 3 (ML-Clustering):** This final process involves merge all the aggregated article tables and gathering all the property headers (e.g. compression strength, Yield Stress) from this final table. We then use a ML-clustering algorithm to group property headers based on their lexical similarity and then assigning uniform labels to each cluster (a new property name). We perform this step because property headers are uniquely named across each research article, so there is a need to keep the naming consistent within our database.

In [331]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, OPTICS, Birch
from scipy.cluster.hierarchy import dendrogram
import nltk, gensim
#nltk.download('punkt')
''' resources = https://www.kaggle.com/reiinakano/basic-nlp-bag-of-words-tf-idf-word2vec-lstm'''

def ml_clustering_headers(df, algo = 'agglom', distance_threshold = 0.5, plot_dendo  = True, k = 20, num_feature = 10):
  ''' this function runs a clustering algorithm on the table headers to derive a list of synonyms for properties extracted'''
  ''' this function '''

  '''    **Parameters**

        1: *str*
            A string containing the path of the article to be extracted.
        
        2: *int* or 'All'
            The specific ID of table to be extracted or extract 'All' the tables
      
      **Returns**
      A list of table properties (table title, pandas Dataframe, table headers)
  '''
  properties = df.columns.values.astype("U")
  new_properties = []
  units_properties = []
  avg_properties = []
  to_omit = ['COMPOSITION', 'CODE/MIX TAG']
  for prop in properties:
    if not any(omit in prop for omit in to_omit):
      prop = prop.split('PROPERTY:')[-1]
      new_prop, unit = remove_unit(prop)
      new_properties.append(new_prop)
      units_properties.append(unit)

  properties = []
  for i,j,k in zip(new_properties, units_properties, avg_properties):
    properties.append(i +' ' + j +' '+str(k))

  properties = new_properties#.astype("U")


  ## Data preprocessing

  from pint import UnitRegistry
  ureg = UnitRegistry()
  check = 10 * ureg('Pa')

  ##change text data into unicode and getting feature vectors

  ##Bag of words
  vectorizer = CountVectorizer(stop_words='english')
  features = vectorizer.fit_transform(properties) 

  ##Tdfif vectorizer - Term Frequency-Inverse Document Frequency (TF-IDF)
  vectorizer = TfidfVectorizer(stop_words = 'english')
  features = vectorizer.fit_transform(properties)
  ##print(type(features))

  ##word2vec features

  w2vec_model = gensim.models.word2vec.Word2Vec(properties, size=300,   
            window=8, min_count=1, sg=1, iter=30)
  mean_embedding_vectorizer = MeanEmbeddingVectorizer(w2vec_model)
  #features = mean_embedding_vectorizer.fit_transform(properties) 
  ##print(features)
  if algo == 'k_means':
    model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
    plot_dendo = False  
  elif algo == 'agglom':
    model =  AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold)
    features = features.toarray()
  elif algo == 'DBSCAN':
    model = DBSCAN(eps = 1.0, min_samples=1)
    plot_dendo = False
  elif algo == 'OPTICS':
    model = OPTICS(min_samples =1)
    features = features.toarray()
    plot_dendo = False
  elif algo == 'Birch':
    model = Birch(threshold = distance_threshold+0.0, n_clusters=None)
    features = features.toarray()
    plot_dendo = False

  model.fit(features)
  
  if plot_dendo:
    plot_dendrogram(model)
  

  new_df = pd.DataFrame(columns = ['properties', 'cluster', 'terms'])
  new_df['cluster'] = model.labels_
  new_df['properties'] = properties

  # order_centroids = model.cluster_centers_.argsort()[:, ::-1]
  terms = vectorizer.get_feature_names()

  new_df.sort_values(['cluster'], inplace = True)
  #display(new_df)

  df_dict = {}
  number_of_headers = len(new_df)
  number_of_cluster = int(max(new_df['cluster']))
  for i in range(number_of_cluster):
    holder = new_df.loc[new_df['cluster'] == i]
    ##print(holder)
    df_dict[str(i)] = list(holder['properties']) 
  ##print(df_dict)
  
  final_df = pd.DataFrame.from_dict(df_dict, orient = 'index')
  ##display(final_df)
  f_name = str(number_of_headers)+'_headers_'+str(number_of_cluster)+'_clusters_birch-threshold_'+str(distance_threshold)+'.csv'
  final_df.to_csv('/content/drive/My Drive/'+f_name, index = True)
  
  
  #test = ['Tensile']
  #vectorizer = TfidfVectorizer(stop_words = 'english')
  test = vectorizer.transform(['Yield Paste 32 Stress']).toarray()
  ##print(features, test)
  ##print(model.predict(test), 'HERE')

  derived_df = pd.read_csv('/content/drive/My Drive/'+f_name)
  derived_df.rename(columns={ derived_df.columns[0]: "Cluster" }, inplace = True)

  #display(derived_df)


  #display(derived_df)
  return new_df



# A method for generating dendrogram
def plot_dendrogram(model, **kwargs):
    from scipy.cluster.hierarchy import dendrogram
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_, counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)

class MyTokenizer:
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        transformed_X = []
        for document in X:
            tokenized_doc = []
            for sent in nltk.sent_tokenize(document):
                tokenized_doc += nltk.word_tokenize(sent)
            transformed_X.append(np.array(tokenized_doc))
        return np.array(transformed_X)
    
    def fit_transform(self, X, y=None):
        return self.transform(X)

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(word2vec.wv.syn0[0])

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = MyTokenizer().fit_transform(X)
        
        return np.array([
            np.mean([self.word2vec.wv[w] for w in words if w in self.word2vec.wv]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
    def fit_transform(self, X, y=None):
        return self.transform(X)


**Selecting 'ScienceDirect' files** 

In [332]:
import os
header_path = '/content/drive/My Drive/SERI_AI_Concrete/data_extraction/html/'
list_of_html_files = os.listdir(header_path)
content_of_html_files = []
new_list_of_html_files = []
for i in list_of_html_files:
  if '.html' in i and 'ScienceDirect' in i:
    contents = open(header_path+i, 'r').read()
    content_of_html_files.append(contents)
    new_list_of_html_files.append(i)
list_of_html_files = new_list_of_html_files[:] 

**Processing a single Article**

In [335]:
  article = list_of_html_files[1] 

  article_name = article.split('.html')[0]
  #print(article)
  article, article1 = open(header_path+article, 'r'), open(header_path+article, 'r')  
  #print(article)
  doi = get_doi(article1)
  #print(doi)
  hp = HTMLTableParser()
  #
  
  hp_table = hp.parse_html_file(article, table_id ='All')

  # hp_table = hp.parse_html_file(article, table_id =4)  
  # #display(hp_table[1])

  
  article_table = ExtractDF([hp_table], article_name, doi).check_improper_table_heading()
  mixtures, tab_titles = ExtractDF([article_table], article_name, doi).make_table()
  
  ##display(mixtures)
  
  #clusters = ml_clustering_headers(mixtures, algo = 'Birch', k = 30) ## do not cluster for composition properties
  ##print('COL/MAX number of clusters', len(clusters), '/',max(clusters['cluster'])+1)
  
  compositions = ExtractDF([article_table], article_name, doi).extract_compositions()
  
  ##display(compositions)
  
  mixtures = pandas_should_have_this(mixtures, tab_titles)
  new_df = join_comp_with_mix(mixtures,  compositions)
  #n_df = pandas_should_have_this(new_df, tab_titles)
  ##display(n_df)
  
  ##display(new_df)
  n_df = prettify(new_df)
  
  display(n_df)
  
  #ml_clustering_headers(n_df, k = 30)
  #n_df = filter_prop(n_df)

  #print(doi)

  #ml_clustering_headers(n_df)

,Article Title,CODE/MIX TAG-1,COMPOSITION: Al2O3 in/for Cement,COMPOSITION: Al2O3 in/for Fly ash,COMPOSITION: Al2O3 in/for Slag,COMPOSITION: CaO in/for Cement,COMPOSITION: CaO in/for Fly ash,COMPOSITION: CaO in/for Slag,COMPOSITION: Fe2O3 in/for Cement,COMPOSITION: Fe2O3 in/for Fly ash,COMPOSITION: Fe2O3 in/for Slag,COMPOSITION: Loss on ignition in/for Cement,COMPOSITION: Loss on ignition in/for Fly ash,COMPOSITION: Loss on ignition in/for Slag,COMPOSITION: MgO in/for Cement,COMPOSITION: MgO in/for Fly ash,COMPOSITION: MgO in/for Slag,COMPOSITION: SO3 in/for Cement,COMPOSITION: SO3 in/for Fly ash,COMPOSITION: SO3 in/for Slag,COMPOSITION: SiO2 in/for Cement,COMPOSITION: SiO2 in/for Fly ash,COMPOSITION: SiO2 in/for Slag,DOI,PROPERTY: CONCRETE/PASTE/MORTAR,PROPERTY: Cement(%),PROPERTY: Coarse Aggregate(%),PROPERTY: Compressive Strength /28 d(MPa),PROPERTY: Compressive Strength /3 d(MPa),PROPERTY: Fine Aggregate(%),PROPERTY: Fly Ash(%),PROPERTY: Paste(%),PROPERTY: Plastic Viscosity (Pa.s),PROPERTY: Slag(%),PROPERTY: Slump (mm),PROPERTY: Table Entry,PROPERTY: Yield Stress (Pa)
0,Mixture design of concrete using simplex centr...,1,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,33.0,41.36,22.89,23.0,NaN,44.0,10.72,NaN,230.0,[2][3],120.29
1,Mixture design of concrete using simplex centr...,2,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,33.0,44.58,25.81,35.0,NaN,32.0,29.18,NaN,95.0,[2][3],555.18
2,Mixture design of concrete using simplex centr...,3,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,42.0,44.54,24.83,24.5,NaN,33.5,28.64,NaN,125.0,[2][3],364.71
3,Mixture design of concrete using simplex centr...,4,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,33.0,42.85,24.95,29.0,NaN,38.0,15.53,NaN,205.0,[2][3],223.11
4,Mixture design of concrete using simplex centr...,5,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,39.0,41.67,24.47,23.0,NaN,38.0,21.51,NaN,195.0,[2][3],247.86
5,Mixture design of concrete using simplex centr...,6,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,39.0,43.11,23.17,29.0,NaN,32.0,24.91,NaN,155.0,[2][3],312.25
6,Mixture design of concrete using simplex centr...,7,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,37.0,40.57,22.14,27.0,NaN,36.0,21.79,NaN,165.0,[2][3],288.50
7,Mixture design of concrete using simplex centr...,8,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,35.0,40.46,23.38,25.0,NaN,40.0,16.40,NaN,200.0,[2][3],145.68
8,Mixture design of concrete using simplex centr...,9,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,NaN,35.0,40.28,24.95,31.0,NaN,34.0,18.79,NaN,160.0,[2][3],339.42
9,Mixture design of concrete using simplex centr...,I,4.58,31.31,13.16,62.13,10.19,38.62,3.27,5.47,1.03,1.86,5.92,0.36,3.13,0.93,10.85,2.8,1.13,3.14,20.76,46.21,31.64,10.1016/j.cemconcomp.2018.03.001,CONCRETE,100,NaN,47.70,25.13,NaN,0,NaN,23.56,0,175.0,[2][3],286.58


**Aggregating all articles** 

In [336]:
all_df = 0
bad_mix = 0 
bad_comp = 0 
count_sci = 0
count_succ = 0
for article in list_of_html_files:  
  if 'ScienceDirect' in article:
    count_sci += 1
    try:
      article_name = article.split('.html')[0]
      #print(article)
      article, article1 = open(header_path+article, 'r'), open(header_path+article, 'r')  
      doi = get_doi(article1)
      hp = HTMLTableParser()

      hp_table = hp.parse_html_file(article, table_id ='All')
      #hp_table = hp.parse_html_file(article, table_id =1)
      
      article_table = ExtractDF([hp_table], article_name, doi).check_improper_table_heading()
      mixtures, tab_titles = ExtractDF([article_table], article_name, doi).make_table()
      if len(mixtures) == 0:
        bad_mix += 1
      #mixtures
      #article_table
      compositions = ExtractDF([article_table], article_name, doi).extract_compositions()
      if len(compositions) == 0:
        bad_comp += 1
      ##display(compositions)
      ##display(mixtures)
      # new_df = join_comp_with_mix(mixtures,  compositions)

      # n_df = pandas_should_have_this(new_df, tab_titles)
      # n_df = prettify(n_df)

      mixtures = pandas_should_have_this(mixtures, tab_titles)
      new_df = join_comp_with_mix(mixtures,  compositions)      
      
      n_df = prettify(new_df)

      try:
        if all_df == 0:
          all_df = n_df
          mix_df = mixtures
      except:
        try:
          all_df = all_df.append(n_df, sort = False, ignore_index = True)
          mix_df = mix_df.append(mixtures, sort = False, ignore_index = True)
        except:
          all_df = all_df.append(n_df, sort = False, ignore_index = True)
          mix_df = mix_df.append(mixtures, sort = False, ignore_index = True)

          pass
      count_succ += 1
    except:
      pass  
all_df.sort_index(axis=1, inplace=True)
thresh = [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
thresh = [0.5]
for i in thresh:
  clusters = ml_clustering_headers(mix_df, algo = 'Birch', distance_threshold =i) ## do not cluster for composition properties
  #print('COL/MAX number of clusters', len(clusters), '/',max(clusters['cluster'])+1)

display(all_df)

#print(count_succ, count_sci)
all_df.sort_index(axis=1, inplace=True)
#all_df.columns[421]
all_df.columns
print('SUCCESSFULLY EXTRACTED FILES', count_succ, 'out of', count_sci)
#len(all_df.columns)
# ## Next step is getting rid of repeated column entries

,Mixture ID,Fitted ηθ for flexural strength,Fitted ηθ for tensile strength,Evaluated ηθ from image analysis
1,SF0-2,0.80,0.56,0.35
2,SF5-2,0.43,NaN,NaN
3,SF10-2,0.57,NaN,NaN
4,SF15-2,0.62,NaN,NaN
5,SF20-2,0.52,NaN,NaN
6,SF25-2,0.45,NaN,NaN


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: Deprec

,Fly ash source,Particle density (g/cm3),SSA (m2/g),IPV,Glassy phase (%),[5]Fe2 + (mol),[5]Fe3 + (mol),cNM (mol),nNF (mol),I,28-day fc (MPa),Notes
1,Collie (West Australian),2.40,0.660,0.470,52.420,0.015,0.062,0.356,0.687,0.79,53 ± 10,Ref. [18]fc was determined on the system with ...
2,2.02,0.46,0.434,60.700,0.006,0.007,0.185,0.920,0.210,33 ± 8,NaN,NaN
3,2.00,0.50,0.431,49.770,0.001,0.005,0.041,0.796,0.060,26 ± 4,NaN,NaN
4,Gladstone (Queensland),2.20,1.000,0.471,76.870,0.013,0.032,0.378,1.152,0.70,19 ± 2,"Ref. [20]Density, SSA and IPV are calculated b..."
5,2.00,0.50,0.416,67.500,0.005,0.012,0.325,1.007,0.390,7.5 ± 0.5,NaN,NaN
6,1.60,0.38,0.391,67.360,0.002,0.010,0.053,1.113,0.050,4.5 ± 1,NaN,NaN
7,FA1 (South Korea),2.00,0.800,0.484,71.400,0.007,0.023,0.229,1.100,0.34,8.7 ± 2.2,"Ref. [53]Density, SSA and IPV are estimated ba..."
8,2.25,1.10,0.508,80.800,0.016,0.000,0.676,1.060,1.230,41.6 ± 12,NaN,NaN


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:205: Deprec

,C0 (MPa),I1 (MPa),P1 (MPa),P2 (μm),H1 (MPa),H2 (MPa),H3 (μm)
2,78.0,−66.8,12.9,10.59,24.0,33.8,11.02
3,78.0-1,−82.3,12.9-1,26.80,NaN,NaN,NaN
4,78.0-2,−94.8,12.9-2,10.10,NaN,NaN,NaN
5,78.0-3,−103.8,12.9-3,0.00,NaN,NaN,NaN


...

,properties,cluster,terms
1,Flow spread,0,NaN
2,Flow rate,1,NaN
76,Flow value,2,NaN
140,Flow time (s),3,NaN
18,Slump,3,NaN
...,...,...,...
65,Quantity /mix NC,145,NaN
66,Quantity /mix VM,145,NaN
75,Yield stress (kPa),146,NaN
106,Yield load (N),147,NaN


,Cluster,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,Flow spread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Flow rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Flow value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Flow time (s),Slump,Slump-flow,Slump flow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Flow energy Y.Sa (N.mm),Flow energy Ub (N.mm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,143,Silica fume (x5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,144,Dynamic yield stress τ0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,145,Quantity /mix NC,Quantity /mix VM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,146,Yield stress (kPa),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Cluster,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0,Flow spread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Flow rate,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Flow value,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Flow time (s),Slump,Slump-flow,Slump flow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Flow energy Y.Sa (N.mm),Flow energy Ub (N.mm),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,143,Silica fume (x5),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,144,Dynamic yield stress τ0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
145,145,Quantity /mix NC,Quantity /mix VM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146,146,Yield stress (kPa),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Article Title,CODE/MIX TAG-1,CODE/MIX TAG-2,CODE/MIX TAG-3,CODE/MIX TAG-4,COMPOSITION: Al2O3 in/for Cement,COMPOSITION: Al2O3 in/for Fly ash,COMPOSITION: Al2O3 in/for Limestone,COMPOSITION: Al2O3 in/for Sand,COMPOSITION: Al2O3 in/for Silica,COMPOSITION: Al2O3 in/for Slag,COMPOSITION: BaO in/for Cement,COMPOSITION: BaO in/for Sand,COMPOSITION: C2S in/for Cement,COMPOSITION: C3A in/for Cement,COMPOSITION: C3S in/for Cement,COMPOSITION: C4AF in/for Cement,COMPOSITION: CaO in/for Cement,COMPOSITION: CaO in/for Fly ash,COMPOSITION: CaO in/for Limestone,COMPOSITION: CaO in/for Sand,COMPOSITION: CaO in/for Silica,COMPOSITION: CaO in/for Slag,COMPOSITION: Cl in/for Cement,COMPOSITION: Cl in/for Sand,COMPOSITION: Cr2O3 in/for Fly ash,COMPOSITION: CuO in/for Fly ash,COMPOSITION: Fe2O3 in/for Cement,COMPOSITION: Fe2O3 in/for Fly ash,COMPOSITION: Fe2O3 in/for Limestone,COMPOSITION: Fe2O3 in/for Sand,COMPOSITION: Fe2O3 in/for Silica,COMPOSITION: Fe2O3 in/for Slag,COMPOSITION: K2O in/for Cement,COMPOSITION: K2O in/for Fly ash,COMPOSITION: K2O in/for Sand,COMPOSITION: K2O in/for Silica,COMPOSITION: K2O in/for Slag,COMPOSITION: LOI in/for Cement,COMPOSITION: LOI in/for Fly ash,...,PROPERTY: Test result/b (h−1),PROPERTY: Test result/df (mm),PROPERTY: Test result/rw (h−1),PROPERTY: Test result/τd (Pa),PROPERTY: Test result/τs (Pa),PROPERTY: Time taken to reach peak temperature (hours),PROPERTY: Total CH(%),PROPERTY: V-Funnel (s),PROPERTY: W(kg/m),PROPERTY: W/CM by mass/ (ratio),PROPERTY: Water,PROPERTY: Water (%) (x4),PROPERTY: Water (g),PROPERTY: Water (kg/m3),PROPERTY: Water weight ratio,PROPERTY: Water(kg/m3),PROPERTY: Water/binder(by cement weight),PROPERTY: Yield Stress (Pa),PROPERTY: Yield load (N),PROPERTY: Yield stress (MPa × 10−3),PROPERTY: Yield stress (Pa),PROPERTY: Yield stress (kPa),PROPERTY: [Na+]/mol/L,PROPERTY: [Si]/mol/L,PROPERTY: a [kPa],PROPERTY: b [min− 1],PROPERTY: b [mm],PROPERTY: c (MPa),PROPERTY: d [mm],PROPERTY: h [mm],PROPERTY: p-value,PROPERTY: st. dev. dry density,PROPERTY: st. dev. strength,PROPERTY: w/c,PROPERTY: w/p ratio (by mass),PROPERTY: w/p ratio (by vol.),PROPERTY: wet density,PROPERTY: α(%),PROPERTY: σ (MPa),PROPERTY: φ (degrees)
0,"Effects of fly ash microsphere on rheology, ad...",M-0-0.6,NaN,NaN,NaN,5.7,26.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Effects of fly ash microsphere on rheology, ad...",M-0-0.7,NaN,NaN,NaN,5.7,26.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Effects of fly ash microsphere on rheology, ad...",M-0-0.8,NaN,NaN,NaN,5.7,26.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.257,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Effects of fly ash microsphere on rheology, ad...",M-0-0.9,NaN,NaN,NaN,5.7,26.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.289,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Effects of fly ash microsphere on rheology, ad...",M-0-1.0,NaN,NaN,NaN,5.7,26.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.7,4.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.6,5.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.321,NaN,NaN,NaN,NaN,NaN,NaN

SUCCESSFULLY EXTRACTED FILES 24 out of 33
